In [1]:
import pandas as pd
from pandas import DataFrame as df

import numpy as np
import re
from datetime import datetime
import dbfread


import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from math import sin,cos,sqrt,atan2,radians 

import requests
import logging
import time
import googlemaps
import reverse_geocoder
import pprint
import folium
import zipfile
import json

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline

matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시

# 한글 깨짐 방지 목적 #
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)
rc('font', family='NanumGothic')

# !pip install fbprophet
# from fbprophet.plot import plot_plotly
# import plotly.offline as py

import pandas_profiling

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 숨길 때

# New way to merge

In [2]:
card = pd.read_csv('Rawdata/카드매출데이터/CARD_SPENDING_190809.txt', sep = "\t")
tp = pd.read_csv('가공데이터/유동_edit1.csv')
tp = tp.iloc[:,1:]

In [3]:
ad = dbfread.DBF('Rawdata/유동인구데이터/행정동경계파일/종로_노원_행정동.dbf')
ad = pd.DataFrame(iter(ad))
ad.head()

,STD_YYYY,HDONG_CD,HDONG_NM,SIDO_CD,SIDO_NM,SGNG_CD,SGNG_NM,AREA,LENGTH,X_COORD,Y_COORD
0,2019,1111051500,청운효자동,11,서울특별시,11110,종로구,2573175.62,7975.05,126.9726940,37.5813060
1,2019,1111053000,사직동,11,서울특별시,11110,종로구,1128161.02,5498.40,126.9709195,37.5734080
2,2019,1111054000,삼청동,11,서울특별시,11110,종로구,1478468.25,7377.76,126.9840280,37.5820830
3,2019,1111055000,부암동,11,서울특별시,11110,종로구,2276588.99,8912.71,126.9665000,37.5899170
4,2019,1111056000,평창동,11,서울특별시,11110,종로구,8951336.18,13312.43,126.9689720,37.6025830


In [4]:
GU = '''
110	515	종로구	청운효자동
110	530	종로구	사직동
110	540	종로구	삼청동
110	550	종로구	부암동
110	560	종로구	평창동
110	570	종로구	무악동
110	580	종로구	교남동
110	600	종로구	가회동
110	615	종로구	종로1.2.3.4가동
110	630	종로구	종로5.6가동
110	640	종로구	이화동
110	650	종로구	혜화동
110	670	종로구	창신1동
110	680	종로구	창신2동
110	690	종로구	창신3동
110	700	종로구	숭인1동
110	710	종로구	숭인2동
350	560	노원구	월계1동
350	570	노원구	월계2동
350	580	노원구	월계3동
350	595	노원구	공릉1동
350	600	노원구	공릉2동
350	611	노원구	하계1동
350	612	노원구	하계2동
350	619	노원구	중계본동
350	621	노원구	중계1동
350	624	노원구	중계4동
350	625	노원구	중계2.3동
350	630	노원구	상계1동
350	640	노원구	상계2동
350	665	노원구	상계3.4동
350	670	노원구	상계5동
350	695	노원구	상계6.7동
350	700	노원구	상계8동
350	710	노원구	상계9동
350	720	노원구	상계10동
'''

df_GU=pd.DataFrame(re.findall(r"[\S]+",GU))
df_GU=pd.DataFrame(df_GU.values.reshape(36,4))
df_GU.rename(columns={0:"GU_CD",1:"DONG_CD",2:"GU",3:"DONG"}, inplace=True)

df_GU['GU_CD']=df_GU['GU_CD'].astype('category')
df_GU['DONG_CD']=df_GU['DONG_CD'].astype('category')
df_GU['GU']=df_GU['GU'].astype('category')
df_GU['DONG']=df_GU['DONG'].astype('category')

In [5]:
list = pd.DataFrame(['숙박(10)', '레저용품(20)', '레저업소(21)', '문화취미(22)', '가구(30)', '전기(31)', '주방용구(32)', '연료판매(33)', '광학제품(34)', '가전(35)', '유통업(40)', '의복(42)', '직물(43)', '신변잡화(44)', '서적문구(50)', '사무통신(52)', '자동차판매(60)', '자동차정비(62)', '의료기관(70)', '보건위생(71)', '요식업소(80)', '음료식품(81)', '수리서비스(92)'], columns=['MCT_CAT_CD'])
cate=list['MCT_CAT_CD'].str.split("(", n=1, expand=True)
cate.rename({0:'NAME',1:'MCT_CAT_CD'},axis=1,inplace=True)
cate['MCT_CAT_CD']=cate['MCT_CAT_CD'].str.replace(")","")
cate['NAME']=cate['NAME'].astype('category')
cate['MCT_CAT_CD']=cate['MCT_CAT_CD'].astype('category')

card.GU_CD = card.GU_CD.astype('str')
card.DONG_CD = card.DONG_CD.astype('str')
df_GU.GU_CD = df_GU.GU_CD.astype('str')
df_GU.DONG_CD = df_GU.DONG_CD.astype('str')

In [6]:
len(cate)

23

In [7]:
card_gd = card.merge(df_GU,how='left', on = ['GU_CD','DONG_CD'])

In [8]:
card_gd.isnull().sum()

STD_DD        0
GU_CD         0
DONG_CD       0
MCT_CAT_CD    0
SEX_CD        0
AGE_CD        0
USE_CNT       0
USE_AMT       0
GU            0
DONG          0
dtype: int64

In [9]:
card_gd.columns = ['STD_YMD', 'GU_CD', 'DONG_CD', 'MCT_CAT_CD', 'SEX_CD', 'AGE_CD',
'USE_CNT', 'USE_AMT', 'GU', 'ADMD_NM']

In [10]:
card_gd.GU = card_gd.GU.astype('str')
card_gd.ADMD_NM = card_gd.ADMD_NM.astype('str')
card_gd.AGE_CD = card_gd.AGE_CD.astype('int64')

In [11]:
card_gd = card_gd[['STD_YMD', 'MCT_CAT_CD', 'SEX_CD', 'AGE_CD','USE_CNT', 'USE_AMT', 'ADMD_NM']]

In [12]:
card_gd.head()

,STD_YMD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,ADMD_NM
0,20180401,21,F,30,4,180,청운효자동
1,20180401,21,F,55,4,22,청운효자동
2,20180401,21,M,20,35,184,청운효자동
3,20180401,21,M,25,70,425,청운효자동
4,20180401,21,M,30,18,82,청운효자동


## 유동인구 데이터를 직관적인 형태로 변형한다.

In [13]:
tp.head()

,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_0004,MAN_0509,MAN_1014,MAN_1519,MAN_2024,MAN_2529,...,TMST_20,TMST_21,TMST_22,TMST_23,SGNG_NM,year,day,month,date,weekday
0,201804,20180401,1111051500,청운효자동,0.05,78.93,403.26,642.66,984.24,1792.30,...,1331.83,1231.07,1041.82,624.62,종로구,2018,1,4,2018-04-01,Sunday
1,201804,20180401,1111053000,사직동,1.25,262.54,1207.22,2177.88,4076.89,6887.72,...,5137.36,4532.06,3589.93,2308.88,종로구,2018,1,4,2018-04-01,Sunday
2,201804,20180401,1111054000,삼청동,0.00,78.07,255.24,331.09,654.32,1264.86,...,875.79,691.84,433.89,229.83,종로구,2018,1,4,2018-04-01,Sunday
3,201804,20180401,1111055000,부암동,0.00,228.50,753.31,981.54,1615.77,2732.00,...,3217.30,2925.42,2374.94,1382.12,종로구,2018,1,4,2018-04-01,Sunday
4,201804,20180401,1111056000,평창동,0.00,354.38,1500.63,1963.08,2593.07,4927.24,...,5661.25,5210.50,4416.75,3060.66,종로구,2018,1,4,2018-04-01,Sunday


In [14]:
# 시간대(TMST_01 ~ TMST_23시)별 유동인구는 제외한다. #
tp = tp.loc[:,~tp.columns[:].isin(tp.columns[34:58])]

In [15]:
tp = tp.loc[:,['STD_YMD','HDONG_NM','MAN_0004', 'MAN_0509',
       'MAN_1014', 'MAN_1519', 'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539',
       'MAN_4044', 'MAN_4549', 'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569',
       'MAN_70U', 'WMAN_0004', 'WMAN_0509', 'WMAN_1014', 'WMAN_1519',
       'WMAN_2024', 'WMAN_2529', 'WMAN_3034', 'WMAN_3539', 'WMAN_4044',
       'WMAN_4549', 'WMAN_5054', 'WMAN_5559', 'WMAN_6064', 'WMAN_6569',
       'WMAN_70U']]

In [16]:
tp.columns

Index(['STD_YMD', 'HDONG_NM', 'MAN_0004', 'MAN_0509', 'MAN_1014', 'MAN_1519',
       'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539', 'MAN_4044', 'MAN_4549',
       'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569', 'MAN_70U', 'WMAN_0004',
       'WMAN_0509', 'WMAN_1014', 'WMAN_1519', 'WMAN_2024', 'WMAN_2529',
       'WMAN_3034', 'WMAN_3539', 'WMAN_4044', 'WMAN_4549', 'WMAN_5054',
       'WMAN_5559', 'WMAN_6064', 'WMAN_6569', 'WMAN_70U'],
      dtype='object')

In [17]:
we = []
we2 = []
for j in range(2,len(tp.columns),1):
    sh1 = tp.columns.str.split('_')[j][0]
    sh2 = tp.columns.str.split('_')[j][1]
    we.append(sh1)
    we2.append(sh2)

In [18]:
we_new = we * (len(np.unique(tp['STD_YMD'])) * len(np.unique(tp['HDONG_NM'])))
we2_new = we2 * (len(np.unique(tp['STD_YMD'])) * len(np.unique(tp['HDONG_NM'])))
trans_pop = tp.iloc[:,2:].values.ravel()
date = [a for i in tp['STD_YMD'] for a in [i] * len(we)]
dong = [uy for j in tp['HDONG_NM'] for uy in [j] * len(we)]

In [19]:
new_tp = pd.DataFrame({'STD_YMD' : date, 'ADMD_NM' : dong, 'SEX_CD' : we_new, 'AGE_CD' : we2_new, 'trans_pop':trans_pop})

## 현재의 테이블 구성[유동 인구데이터 + 카드소비 데이터 ] 목적은 유동인구 데이터와 환경데이터를 통해 카드 소비를 맞추기 위한 것이다. 

In [20]:
new_tp['SEX_CD'].replace({'MAN' : 'M','WMAN':'F'},inplace=True)

In [21]:
new_tp['AGE_CD'].replace({'0004':'20','0509' :'20','1014':'20','1519' :'20',
                          '2024':'20','2529':'25',
                          '3034':'30','3539':'35',
                          '4044':'40','4549':'45',
                          '5054':'50','5559':'55',
                          '6064':'60','6569':'65','70U' : '65'},inplace=True)

In [22]:
new_tp2 = new_tp.groupby(['STD_YMD','ADMD_NM','SEX_CD','AGE_CD'])['trans_pop'].sum().reset_index(level=[0,1,2,3])
new_tp2['AGE_CD'] = new_tp2['AGE_CD'].astype('int64')

In [23]:
new_tp3 = pd.merge(new_tp2,card_gd,on=['STD_YMD','ADMD_NM','SEX_CD','AGE_CD'])

In [24]:
new_tp3.head()

,STD_YMD,ADMD_NM,SEX_CD,AGE_CD,trans_pop,MCT_CAT_CD,USE_CNT,USE_AMT
0,20180401,가회동,F,20,1160.64,10,4,173
1,20180401,가회동,F,20,1160.64,40,101,547
2,20180401,가회동,F,20,1160.64,42,31,477
3,20180401,가회동,F,20,1160.64,44,4,434
4,20180401,가회동,F,20,1160.64,71,13,81


In [25]:
new_tp3.to_csv('가공데이터/merge_유동_카드_edit1.csv',encoding='utf-8')

## 환경데이터를 {유동 + 카드} 데이터와 합친다

In [26]:
air = pd.read_csv('가공데이터/환경_edit1.csv')
air

,date,year,quarter,month,weekday,day,district,gu,p10_most_freq_day,p10_mean_day,...,humi_median_day,humi_25per,humi_75per,temp_mean_day,temp_median_day,temp_25per,temp_75per,noise_mean_day,noise_median_day,dis_index_most_freq2
0,2018-04-01,2018,2,4,Sunday,1,가회동,jonglo,나쁨,66.773843,...,62.138889,55.855556,66.687500,16.518461,17.329722,14.266528,18.746111,52.369792,52.533333,낮음
1,2018-04-02,2018,2,4,Monday,2,가회동,jonglo,나쁨,64.771759,...,61.513889,50.356944,73.081944,19.367951,18.616667,17.217639,21.169583,52.912384,53.191667,낮음
2,2018-04-03,2018,2,4,Tuesday,3,가회동,jonglo,나쁨,63.759838,...,65.633333,58.222222,68.148611,19.069664,18.500833,18.093611,20.568056,53.065972,53.527778,낮음
3,2018-04-04,2018,2,4,Wednesday,4,가회동,jonglo,좋음,18.720500,...,53.291667,46.194444,82.569444,14.161678,14.432778,12.168333,15.800278,53.301505,53.516667,낮음
4,2018-04-05,2018,2,4,Thursday,5,가회동,jonglo,좋음,12.597801,...,86.141667,82.935417,86.795833,8.552627,8.235833,8.072639,8.851944,53.525000,53.958333,낮음
5,2018-04-06,2018,2,4,Friday,6,가회동,jonglo,매우나쁨,82.220023,...,62.447222,49.897222,86.291667,7.215961,7.367778,6.533333,8.291389,53.605324,53.672222,낮음
6,2018-04-07,2018,2,4,Saturday,7,가회동,jonglo,나쁨,40.517824,...,36.877778,31.562500,50.672222,5.183819,4.962222,3.223056,6.587222,53.309491,53.288889,낮음
7,2018-04-08,2018,2,4,Sunday,8,가회동,jonglo,보통,33.939120,...,47.736111,39.736111,77.717361,4.882361,3.908333,3.339167,7.346389,52.797338,53.022222,낮음
8,2018-04-09,2018,2,4,Monday,9,가회동,jonglo,나쁨,41.764352,...,43.755556,29.720833,82.772917,8.720266,10.630278,2.966875,13.205694,52.872917,53.102778,낮음
9,2018-04-10,2018,2,4,Tuesday,10,가회동,jonglo,나쁨,47.675463,...,45.152778,32.509722,51.497222,15.928993,16.712222,10.135417,20.936111,53.785301,54.051389,낮음


In [27]:
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10220 entries, 0 to 10219
Data columns (total 29 columns):
date                    10220 non-null object
year                    10220 non-null int64
quarter                 10220 non-null int64
month                   10220 non-null int64
weekday                 10220 non-null object
day                     10220 non-null int64
district                10220 non-null object
gu                      10220 non-null object
p10_most_freq_day       10220 non-null object
p10_mean_day            10220 non-null float64
p10_25per               10220 non-null float64
p10_median_day          10220 non-null float64
p10_75per               10220 non-null float64
p25_most_freq_day       10220 non-null object
p25_mean_day            10220 non-null float64
p25_25per               10220 non-null float64
p25_median_day          10220 non-null float64
p25_75per               10220 non-null float64
humi_mean_day           10220 non-null float64
humi_median_

In [28]:
air['date'] = pd.to_datetime(air['date'])

In [29]:
new_tp3.rename(columns = {'STD_YMD':'date','ADMD_NM':'district'},inplace=True)

In [30]:
new_tp3['date'] = pd.to_datetime(new_tp3['date'],format = '%Y%m%d')

In [31]:
mvp = pd.merge(air,new_tp3,on=['date','district'])

{환경 + 유동 + 카드} data가 26개 동만 남음이 확인 되었다.

In [32]:
len(np.unique(mvp['district']))

26

In [33]:
mvp.to_csv('가공데이터/merge_카드_유동_환경_edit1.csv',encoding='utf-8')

In [34]:
mvp['dis_index_most_freq2'].value_counts()

낮음      1194241
보통       285865
매우높음     138556
높음        89508
Name: dis_index_most_freq2, dtype: int64